<center><h1>Step 0 - Preprocessing</h1></center> 

In this section, we first read the data including the bug reports and source code files of all 51 projects and for ease of access, we save them as two pickle files in the ./Data directory. Therefore, this set of code will populate the ./Data directory with "allBugReports.pickle" which is a pandas dataframe that contains all the bug reports from all projects and "allSourceCodes.pickle" which is a pandas dataframe that contains all source files after preprocessing.

### Required Libraries

In [1]:
from __future__ import division
import pandas as pd
# import ray.data as ray_pd
import modin.pandas as ray_pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
import warnings
import javalang
import re
import glob
import math
import time
from scipy import spatial
import scipy.spatial.distance
import xml.etree.ElementTree as ET
import requests
import multiprocessing
from tqdm import tqdm_notebook
from time import gmtime, strftime
from random import randint
warnings.simplefilter(action='ignore', category=FutureWarning)
from pathlib import Path
import zlib
import pathlib
import time

In [2]:
import ray

In [3]:
ray.init()

2022-02-25 15:10:52,474	ERROR services.py:1254 -- Failed to start the dashboard: Failed to read dashbord log: [Errno 2] No such file or directory: '/tmp/ray/session_2022-02-25_15-10-31_279754_18960/logs/dashboard.log'


{'node_ip_address': '172.16.138.255',
 'raylet_ip_address': '172.16.138.255',
 'redis_address': '172.16.138.255:6379',
 'object_store_address': '/tmp/ray/session_2022-02-25_15-10-31_279754_18960/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-02-25_15-10-31_279754_18960/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-02-25_15-10-31_279754_18960',
 'metrics_export_port': 60327,
 'node_id': 'c834043192deff829aca70f183500955aab1e8befbf9d48fd5668232'}

<center><h1>Splitting code and natural language</h1></center> 

In [4]:

def split_natural_lang(doc):
    """
    @Receives: a document in natural language (bugreport)
    @Process: splits it as described in BugLocator
    @Return: a list of lower cased words
    """
    try:
        wordList=[]
        word=''
        for char in doc:
            if char.isalnum() or char=='\'':
                word+=char
            else:
                if len(word)>0:
                    wordList.append(word)
                    word=''
        if len(word)>0:
            wordList.append(word)
        return wordList
    except:
        return []


def code_splitter(sourceCode):
    """
    @Receives: a code
    @Process: splits it same as described in BugLocator
    @Return: a list of lower cased words
    """
    contentBuf = []
    wordBuf = []
    for char in sourceCode:
        if ((char >= 'a' and char <= 'z') or (char >= 'A' and char <= 'Z')):
            wordBuf.append(char)
            continue
        length = len(wordBuf)
        if (length != 0):
            k = 0
            for i in range(length-1):
                j=i+1
                first_char = wordBuf[i]
                second_char = wordBuf[j]
                if ((first_char >= 'A' and first_char <= 'Z') and (second_char >= 'a' and second_char <= 'z')):
                    contentBuf.append(wordBuf[k:i])
                    contentBuf.append(' ')
                    k = i
                    continue
                if ((first_char >= 'a' and first_char <= 'z') and (second_char >= 'A' and second_char <= 'Z')):
                    contentBuf.append(wordBuf[k:j])
                    contentBuf.append(' ')
                    k = j
                    continue
            if (k < length):
                contentBuf.append(wordBuf[k:])
                contentBuf.append(" ")
            wordBuf=[]
    words=''
    for each in contentBuf:
        if isinstance(each,str):
            words+=each
        else: 
            for term in each: 
                words+=term
    words= words.split()
    contentBuf = []
    for i in range(len(words)):
        if (words[i].strip()!="" and len(words[i]) >= 2):
            contentBuf.append(words[i])
    return contentBuf



def general_preprocessor(doc,mode):
    """
    @Receives: a document (code or bug report denoted by mode)
    @Process: processes the docucument by stemming and removing stop-words and converting to lower cases
    @Return: a list of lower cased words
    """
    JavaKeywords=["abstract", "continue", "for", 
                "new", "switch", "assert", "default", "goto", "package", 
                "synchronized", "boolean", "do", "if", "private", "this", 
                "break", "double", "implements", "protected", "throw", "byte", 
                "else", "import", "public", "throws", "case", "enum", 
                "instanceof", "return", "transient", "catch", "extends", "int", 
                "short", "try", "char", "final", "interface", "static", "void", 
                "class", "finally", "long", "strictfp", "volatile", "const", 
                "float", "native", "super", "while", "org", "eclipse", "swt", 
                "string", "main", "args", "null", "this", "extends", "true", 
                "false"]
    stop_words=["a", "a's", "able", "about", "above",
                "according", "accordingly", "across", "actually", "after",
                "afterwards", "again", "against", "ain't", "all", "allow",
                "allows", "almost", "alone", "along", "already", "also",
                "although", "always", "am", "among", "amongst", "an", "and",
                "another", "any", "anybody", "anyhow", "anyone", "anything",
                "anyway", "anyways", "anywhere", "apart", "appear",
                "appreciate", "appropriate", "are", "aren't", "around", "as",
                "aside", "ask", "asking", "associated", "at", "available",
                "away", "awfully", "b", "be", "became", "because", "become",
                "becomes", "becoming", "been", "before", "beforehand",
                "behind", "being", "believe", "below", "beside", "besides",
                "best", "better", "between", "beyond", "both", "brief", "but",
                "by", "c", "c'mon", "c's", "came", "can", "can't", "cannot",
                "cant", "cause", "causes", "certain", "certainly", "changes",
                "clearly", "co", "com", "come", "comes", "concerning",
                "consequently", "consider", "considering", "contain",
                "containing", "contains", "corresponding", "could", "couldn't",
                "course", "currently", "d", "definitely", "described",
                "despite", "did", "didn't", "different", "do", "does",
                "doesn't", "doing", "don't", "done", "down", "downwards",
                "during", "e", "each", "edu", "eg", "eight", "either", "else",
                "elsewhere", "enough", "entirely", "especially", "et", "etc",
                "even", "ever", "every", "everybody", "everyone", "everything",
                "everywhere", "ex", "exactly", "example", "except", "f", "far",
                "few", "fifth", "first", "five", "followed", "following",
                "follows", "for", "former", "formerly", "forth", "four",
                "from", "further", "furthermore", "g", "get", "gets",
                "getting", "given", "gives", "go", "goes", "going", "gone",
                "got", "gotten", "greetings", "h", "had", "hadn't", "happens",
                "hardly", "has", "hasn't", "have", "haven't", "having", "he",
                "he's", "hello", "help", "hence", "her", "here", "here's",
                "hereafter", "hereby", "herein", "hereupon", "hers", "herself",
                "hi", "him", "himself", "his", "hither", "hopefully", "how",
                "howbeit", "however", "i", "i'd", "i'll", "i'm", "i've", "ie",
                "if", "ignored", "immediate", "in", "inasmuch", "inc",
                "indeed", "indicate", "indicated", "indicates", "inner",
                "insofar", "instead", "into", "inward", "is", "isn't", "it",
                "it'd", "it'll", "it's", "its", "itself", "j", "just", "k",
                "keep", "keeps", "kept", "know", "knows", "known", "l", "last",
                "lately", "later", "latter", "latterly", "least", "less",
                "lest", "let", "let's", "like", "liked", "likely", "little",
                "look", "looking", "looks", "ltd", "m", "mainly", "many", 
                "may", "maybe", "me", "mean", "meanwhile", "merely", "might",
                "more", "moreover", "most", "mostly", "much", "must", "my",
                "myself", "n", "name", "namely", "nd", "near", "nearly",
                "necessary", "need", "needs", "neither", "never",
                "nevertheless", "new", "next", "nine", "no", "nobody", "non",
                "none", "noone", "nor", "normally", "not", "nothing", "novel",
                "now", "nowhere", "o", "obviously", "of", "off", "often", "oh",
                "ok", "okay", "old", "on", "once", "one", "ones", "only",
                "onto", "or", "other", "others", "otherwise", "ought", "our",
                "ours", "ourselves", "out", "outside", "over", "overall",
                "own", "p", "particular", "particularly", "per", "perhaps",
                "placed", "please", "plus", "possible", "presumably",
                "probably", "provides", "q", "que", "quite", "qv", "r",
                "rather", "rd", "re", "really", "reasonably", "regarding",
                "regardless", "regards", "relatively", "respectively", "right",
                "s", "said", "same", "saw", "say", "saying", "says", "second",
                "secondly", "see", "seeing", "seem", "seemed", "seeming",
                "seems", "seen", "self", "selves", "sensible", "sent",
                "serious", "seriously", "seven", "several", "shall", "she",
                "should", "shouldn't", "since", "six", "so", "some",
                "somebody", "somehow", "someone", "something", "sometime",
                "sometimes", "somewhat", "somewhere", "soon", "sorry",
                "specified", "specify", "specifying", "still", "sub", "such",
                "sup", "sure", "t", "t's", "take", "taken", "tell", "tends",
                "th", "than", "thank", "thanks", "thanx", "that", "that's",
                "thats", "the", "their", "theirs", "them", "themselves",
                "then", "thence", "there", "there's", "thereafter", "thereby",
                "therefore", "therein", "theres", "thereupon", "these", "they",
                "they'd", "they'll", "they're", "they've", "think", "third",
                "this", "thorough", "thoroughly", "those", "though", "three",
                "through", "throughout", "thru", "thus", "to", "together",
                "too", "took", "toward", "towards", "tried", "tries", "truly",
                "try", "trying", "twice", "two", "u", "un", "under",
                "unfortunately", "unless", "unlikely", "until", "unto", "up",
                "upon", "us", "use", "used", "useful", "uses", "using",
                "usually", "uucp", "v", "value", "various", "very", "via",
                "viz", "vs", "w", "want", "wants", "was", "wasn't", "way",
                "we", "we'd", "we'll", "we're", "we've", "welcome", "well",
                "went", "were", "weren't", "what", "what's", "whatever",
                "when", "whence", "whenever", "where", "where's", "whereafter",
                "whereas", "whereby", "wherein", "whereupon", "wherever",
                "whether", "which", "while", "whither", "who", "who's",
                "whoever", "whole", "whom", "whose", "why", "will", "willing",
                "wish", "with", "within", "without", "won't", "wonder",
                "would", "would", "wouldn't", "x", "y", "yes", "yet", "you",
                "you'd", "you'll", "you're", "you've", "your", "yours",
                "yourself", "yourselves", "z", "zero","quot"]
    
    porter = PorterStemmer()
    Java_keyWords=[porter.stem(each.strip().lower()) for each in JavaKeywords]
    natural_stop_words=[porter.stem(each.strip().lower()) for each in stop_words]
    
    processed_doc=[]
    if mode=="code":
        splitted_doc=[porter.stem(term.lower()) for term in code_splitter(doc)]
        processed_doc=[term for term in splitted_doc if not(term in Java_keyWords or
                                                            term in natural_stop_words or len(term)<2)]
    elif mode=="text":
        splitted_doc=[porter.stem(term.lower()) for term in split_natural_lang(doc)]
        processed_doc=[term for term in splitted_doc if not(term in natural_stop_words or len(term)<2)]
    return processed_doc

<center><h1>Loading source codes into pandas Dataframe</h1></center> 

In [5]:
def classNames_methodNames(node):
    result=''
    if isinstance(node,javalang.tree.MethodDeclaration) or isinstance(node,javalang.tree.ClassDeclaration):
        return node.name.lower()+' '
    if not (isinstance(node,javalang.tree.PackageDeclaration) or
        isinstance(node,javalang.tree.FormalParameter) or
       isinstance(node,javalang.tree.Import)):
        if node:
            if isinstance(node, javalang.ast.Node):
                for childNode in node.children:
                    result+=classNames_methodNames(childNode)
    return result
    
def traverse_node(node,i=0):
    i+=1
    result=''
    if not(isinstance(node,javalang.tree.PackageDeclaration)
            or isinstance(node,javalang.tree.FormalParameter)            
            or isinstance(node,javalang.tree.Import)
            or isinstance(node,javalang.tree.CompilationUnit)):
        if node:
            if (isinstance(node,int) or isinstance(node,str) or isinstance(node,float)) and i==2:
                result+=node+' '
            if isinstance(node, javalang.ast.Node):
                for childNode in node.children:
                    result+=traverse_node(childNode,i)
    return result

def code_parser(code):
    try:
        tree = javalang.parse.parse(code)
        return ''.join([traverse_node(node) for path, node in tree]) + ' ' + ''.join([classNames_methodNames(node)
                                                                                      for path, node in tree])
    except Exception as e: 
        print(e)
        return ''

    
@ray.remote
def async_sourcecode_procesing(source_code):
#     print(source_code)
#     try:
    sc_code = zlib.decompress(bytes.fromhex(source_code)).decode()
#     except:
#         print(source_code)
# #     print(sc_code)
    processed_code=general_preprocessor(code_parser(sc_code),'code')
#     print(processed_code)
    return zlib.compress(" ".join(processed_code).encode("utf-8")).hex()

<center><h1>Loading bug reports pandas Dataframe</h1></center> 

In [6]:
@ray.remote
def loadBugs2df(report):
#     print(report)
    return general_preprocessor(report,"text")


In [7]:
def get_combined_full_dataset(project_name, test_percentage=0.4):
    df1 = pd.read_csv("{}.csv".format(project_name), delimiter='\t')
    df2 = pd.read_csv("{}_features.csv".format(project_name))
    df3 = pd.read_csv("{}_features_file_content.csv".format(project_name))
    df4 = pd.merge(df2, df3, left_on='cid', right_on='cid', how='inner')
    df5 = pd.merge(df1, df4, left_on='id', right_on='report_id', how='inner')
    df5['report'] = df5['summary'] + df5['description']
    df5['project_name'] = project_name.split("/")[-1]
    train_pos, test_pos = train_test_split(df5[df5['match'] == 1], test_size=test_percentage, random_state=13, shuffle=False)
    train, test = df5[df5['bug_id'].isin(train_pos['bug_id'])], df5[df5['bug_id'].isin(test_pos['bug_id'])]
    train = train.copy().reset_index(drop=True)
    small_train = pd.DataFrame(columns=train.columns)
    for item in train['bug_id'].unique():
        temp = pd.concat((train[(train['bug_id'] == item) & (train['match'] == 1)],
                          train[(train['bug_id'] == item) & (train['match'] == 0)].head(10)))
        small_train = pd.concat((small_train, temp))
    small_train.drop(columns=set(small_train.columns) - {'id', 'cid', 'report', 'file_content', 'match'}, inplace=True)
    return small_train

In [8]:
def create_random_dataset(dataset_list, full_size=1000):
    part_size = 1.0 / len(dataset_list)
    temp_df = pd.DataFrame(columns=dataset_list[0].columns)
    for item in dataset_list:
        temp_df = temp_df.append(
            item.sample(frac=(full_size * part_size) / len(item), replace=False, random_state=13).reset_index(
                drop=True)).reset_index(drop=True)
    return temp_df.sample(frac=1, random_state=13).reset_index(drop=True)


<center><h1>Main Preprocessing class</h1></center> 

In [9]:
class PreprocessingUnit:

    # all_projects_source_codes=pd.DataFrame([])
    # all_projects_bugreports=pd.DataFrame([])
    
    

    def __init__(self,project_name):
#         project_df = ray_pd.DataFrame([])
        self.dataPath=project_name
        # self.dataFolder=os.path.join(os.getcwd(),'Data')
        # if not os.path.exists(self.dataFolder):
        #     os.makedirs(self.dataFolder)
            
    def execute(self):

        self.loadEverything()

    def loadEverything(self):
#         vectorize=False
#         if PreprocessingUnit.all_projects_bugreports.empty:
#             bugReportFile=os.path.join(self.dataFolder,'allBugReports.csv')
#             if not os.path.isfile(bugReportFile):
#                 PreprocessingUnit.all_projects_bugreports=load_all_BRs(dataPath=self.dataPath)
#                 vectorize=True
#                 if vectorize:
#                     PreprocessingUnit.all_projects_bugreports.to_csv(bugReportFile, index=False)
#             else: 
#                 PreprocessingUnit.all_projects_bugreports=pd.read_pickle(bugReportFile)
# #         if vectorize:
# #             with open()
#         print("*** All bug reports are are preprocessed and stored as: {} ***".format('/'.join(bugReportFile.split('/')[-2:])))
        start = time.time()
        df1 = ray_pd.read_csv("{}.csv".format(self.dataPath), delimiter='\t')
        df2 = ray_pd.read_csv("{}_features.csv".format(self.dataPath))
        df3 = ray_pd.read_csv("{}_features_file_content.csv".format(self.dataPath))
        df4 = ray_pd.merge(df2, df3, left_on='cid', right_on='cid', how='inner')
        df5 = ray_pd.merge(df1, df4, left_on='id', right_on='report_id', how='inner')
        df5['report'] = df5['summary'] + df5['description']
        df5['project_name'] = self.dataPath.split("/")[-1]
        df5 = df5.head()
#         modin_df = from_pandas(df5)
        
#         df5 = df5.head()
#         ray_pd = ray_pd.from_pandas(df5)
        reports = df5.report.values.tolist()
        source_codes = df5.file_content.values.tolist()
        processed_reports = []
        processed_source_codes = []
        for i, r in enumerate(reports):
            if len(processed_reports) > 1000:
                num_ready = i-1000
                ray.wait(processed_reports, num_returns=num_ready)

            processed_reports.append(loadBugs2df.remote(r))
#             try:
#                 processed_reports.append(loadBugs2df(r))
#             except:
#                 processed_reports.append(None)
        for i, sc in enumerate(source_codes):
            if len(processed_source_codes) > 1000:
                    num_ready = i-1000
                    ray.wait(processed_source_codes, num_returns=num_ready)
            processed_source_codes.append(async_sourcecode_procesing.remote(sc))
# #             try:
# #                 processed_source_codes.append(async_sourcecode_procesing(sc))
# #             except:
# #                 processed_reports.append(None)
        processed_reports_ = []
        for pr in processed_reports:
            try:
                processed_reports_.append(ray.get(pr))
            except:
                processed_reports_.append(ray.get(None))
        processed_reports = processed_reports_
        processed_src_ = []
        for pr_src in processed_source_codes:
            try:
                processed_src_.append(ray.get(pr_src))
            except:
                processed_src_.append(ray.get(None))
        processed_reports = processed_reports_
        processed_source_codes = processed_src_
        df5["globug_processed_report"] = processed_reports #df5.report.apply(lambda x: loadBugs2df.remote(x))
        df5["globug_processed_file_content"] = processed_source_codes #df5.file_content.apply(lambda x: async_sourcecode_procesing.remote(x))

        processed_folder = os.path.join("/".join(self.dataPath.split("/")[:-2]), "processed_bl_reports")
        pathlib.Path(processed_folder).mkdir(parents=True, exist_ok=True)

        project = self.dataPath.split("/")[-1]
        project_processed_folder = os.path.join(processed_folder, project)
        pathlib.Path(project_processed_folder).mkdir(parents=True, exist_ok=True)
        project_processed_file = os.path.join(project_processed_folder, f"{project}_processed.csv")

        df5.to_csv(project_processed_file, index=False)
        end = time.time()
        print(f"Processing project: {project} completed")
            
        

In [10]:
@ray.remote
def process_df(path):
    pu = PreprocessingUnit(path)
    pu.execute()
    return pu

In [11]:
projects = ["Birt", "Tomcat", "JDT", "SWT", "Eclipse_Platform_UI"]
# projects = ["AspectJ"]

In [ ]:
processing = []
for p in projects:
    data_path = (f"/home/varumuga/scratch/thesis/replication/bench_bl_dataset/Dataset/{p}")
    processing.append(process_df.remote(data_path))

processing = ray.get(processing) 

(pid=2259) UserWarning: `read_*` implementation has mismatches with pandas:
(pid=2259) Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.
(pid=2282) UserWarning: `read_*` implementation has mismatches with pandas:
(pid=2282) Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.
(pid=2323) UserWarning: `read_*` implementation has mismatches with pandas:
(pid=2323) Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.
(pid=2274) UserWarning: `read_*` implementation has mismatches with pandas:
(pid=2274) Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.
(pid=2291) UserWarning: `read_*` implementation has mismatches with pandas:
(pid=2291) Data types of partitions are different! Pleas

### MAIN

In [ ]:
# df = pd.read_csv("/home/varumuga/scratch/thesis/replication/bench_bl_dataset/processed_bl_reports/AspectJ_processed.csv")

In [ ]:
# type(preprocessor.all_projects_bugreports)

In [ ]:
# preprocessor.dataFolder
# bugReportFile=os.path.join(preprocessor.dataFolder,'allBugReports.csv')

In [ ]:
# bugReportFile

In [ ]:
# PreprocessingUnit.all_projects_bugreports.to_csv(bugReportFile, index=False)

In [ ]:
# a = pd.read_pickle(bugReportFile)

In [ ]:
# a.head(10)

In [ ]:
# PreprocessingUnit.all_projects_bugreports.head()